In [0]:
dbutils.fs.mount(
    source = "wasbs://retail@retailstorage60.blob.core.windows.net",
    mount_point = "/mnt/retail",
    extra_configs = {
        "fs.azure.account.key.retailstorage60.blob.core.windows.net": "MIjtLuiPbP64h67+V1N1m9QUUsCvNUM4ViR3YiSdO9GO4fgPDaWZbX0kHhTcMtkQK8Ta6pDZYwe1+AStyoWBCg=="
    }
)


In [0]:
dbutils.fs.ls('/mnt/retail/bronze/transaction/')

Out[8]: [FileInfo(path='dbfs:/mnt/retail/bronze/transaction/dbo.transactions.parquet', name='dbo.transactions.parquet', size=1757, modificationTime=1751238377000)]

In [0]:
# Read raw data from Bronze layer

df_transactions = spark.read.parquet('/mnt/retail/bronze/transaction/')
df_products = spark.read.parquet('/mnt/retail/bronze/product/')
df_stores = spark.read.parquet('/mnt/retail/bronze/store/')

df_customers = spark.read.parquet('/mnt/retail/bronze/customer/alicorduk/DataEngineer/refs/heads/main/')
display(df_transactions)



transaction_id,customer_id,product_id,store_id,quantity,transaction_date
1,127,8,4,4,2025-03-31
2,105,3,4,5,2024-11-12
3,116,2,2,3,2025-05-01
4,120,8,1,1,2024-11-02
5,105,5,2,1,2025-03-17
6,110,7,3,5,2025-01-04
7,110,7,2,5,2025-01-01
8,126,7,5,2,2025-06-08
9,123,1,3,2,2024-10-08
10,124,2,2,5,2024-08-27


In [0]:

from pyspark.sql.functions import col

# Convert types and clean data
df_transactions = df_transactions.select(
    col("transaction_id").cast("int"),
    col("customer_id").cast("int"),
    col("product_id").cast("int"),
    col("store_id").cast("int"),
    col("quantity").cast("int"),
    col("transaction_date").cast("date")
)

df_products = df_products.select(
    col("product_id").cast("int"),
    col("product_name"),
    col("category"),
    col("price").cast("double")
)

df_stores = df_stores.select(
    col("store_id").cast("int"),
    col("store_name"),
    col("location")
)

df_customers = df_customers.select(
    "customer_id", "first_name", "last_name", "email", "city", "registration_date"
).dropDuplicates(["customer_id"])




In [0]:
# Join all data
df_silver = df_transactions \
    .join(df_customers, "customer_id") \
    .join(df_products, "product_id") \
    .join(df_stores, "store_id") \
    .withColumn("total_amount", col("quantity") * col("price"))


In [0]:
display(df_silver)

store_id,product_id,customer_id,transaction_id,quantity,transaction_date,first_name,last_name,email,city,registration_date,product_name,category,price,store_name,location,total_amount
5,7,101,28,3,2024-11-15,Ravi,Yadav,user101@example.com,Delhi,2023-09-14,Smartwatch,Electronics,4999.0,Mega Plaza,Chennai,14997.0
3,1,102,11,2,2024-08-11,Nina,Joshi,user102@example.com,Mumbai,2024-01-21,Wireless Mouse,Electronics,799.99,Tech World Outlet,Bangalore,1599.98
4,1,103,18,3,2024-09-05,Sonal,Sharma,user103@example.com,Bangalore,2023-07-10,Wireless Mouse,Electronics,799.99,Downtown Mini Store,Pune,2399.9700000000003
3,3,104,13,4,2025-05-04,Karan,Patel,user104@example.com,Hyderabad,2024-02-05,Yoga Mat,Fitness,499.0,Tech World Outlet,Bangalore,1996.0
3,1,105,21,5,2024-10-02,Riya,Singh,user105@example.com,Chennai,2023-06-28,Wireless Mouse,Electronics,799.99,Tech World Outlet,Bangalore,3999.95
2,5,105,5,1,2025-03-17,Riya,Singh,user105@example.com,Chennai,2023-06-28,Notebook Set,Stationery,149.0,High Street Store,Delhi,149.0
4,3,105,2,5,2024-11-12,Riya,Singh,user105@example.com,Chennai,2023-06-28,Yoga Mat,Fitness,499.0,Downtown Mini Store,Pune,2495.0
3,9,107,22,4,2024-11-16,Priya,Kapoor,user107@example.com,Ahmedabad,2023-05-12,Dumbbell Set,Fitness,1999.0,Tech World Outlet,Bangalore,7996.0
1,5,108,12,4,2025-05-26,Rahul,Verma,user108@example.com,Kolkata,2023-08-19,Notebook Set,Stationery,149.0,City Mall Store,Mumbai,596.0
5,8,109,17,5,2024-07-10,Pooja,Mehta,user109@example.com,Delhi,2024-04-01,Desk Organizer,Accessories,399.0,Mega Plaza,Chennai,1995.0


In [0]:
silver_path = "/mnt/retail/silver/"

df_silver.write.mode("overwrite").format("delta").save(silver_path)


In [0]:
spark.sql(f"""
DROP TABLE IF EXISTS retail_silver_cleaned;
CREATE TABLE retail_silver_cleaned
USING DELTA
LOCATION '/mnt/retail/silver/';
""")


Out[19]: DataFrame[]

In [0]:
%sql select * from retail_silver_cleaned

store_id,product_id,customer_id,transaction_id,quantity,transaction_date,first_name,last_name,email,city,registration_date,product_name,category,price,store_name,location,total_amount
5,7,101,28,3,2024-11-15,Ravi,Yadav,user101@example.com,Delhi,2023-09-14,Smartwatch,Electronics,4999.0,Mega Plaza,Chennai,14997.0
3,1,102,11,2,2024-08-11,Nina,Joshi,user102@example.com,Mumbai,2024-01-21,Wireless Mouse,Electronics,799.99,Tech World Outlet,Bangalore,1599.98
4,1,103,18,3,2024-09-05,Sonal,Sharma,user103@example.com,Bangalore,2023-07-10,Wireless Mouse,Electronics,799.99,Downtown Mini Store,Pune,2399.9700000000003
3,3,104,13,4,2025-05-04,Karan,Patel,user104@example.com,Hyderabad,2024-02-05,Yoga Mat,Fitness,499.0,Tech World Outlet,Bangalore,1996.0
3,1,105,21,5,2024-10-02,Riya,Singh,user105@example.com,Chennai,2023-06-28,Wireless Mouse,Electronics,799.99,Tech World Outlet,Bangalore,3999.95
2,5,105,5,1,2025-03-17,Riya,Singh,user105@example.com,Chennai,2023-06-28,Notebook Set,Stationery,149.0,High Street Store,Delhi,149.0
4,3,105,2,5,2024-11-12,Riya,Singh,user105@example.com,Chennai,2023-06-28,Yoga Mat,Fitness,499.0,Downtown Mini Store,Pune,2495.0
3,9,107,22,4,2024-11-16,Priya,Kapoor,user107@example.com,Ahmedabad,2023-05-12,Dumbbell Set,Fitness,1999.0,Tech World Outlet,Bangalore,7996.0
1,5,108,12,4,2025-05-26,Rahul,Verma,user108@example.com,Kolkata,2023-08-19,Notebook Set,Stationery,149.0,City Mall Store,Mumbai,596.0
5,8,109,17,5,2024-07-10,Pooja,Mehta,user109@example.com,Delhi,2024-04-01,Desk Organizer,Accessories,399.0,Mega Plaza,Chennai,1995.0


In [0]:
# Load cleaned transactions from Silver layer
silver_df = spark.read.format("delta").load("/mnt/retail/silver/")


In [0]:
display(df_silver)

store_id,product_id,customer_id,transaction_id,quantity,transaction_date,first_name,last_name,email,city,registration_date,product_name,category,price,store_name,location,total_amount
5,7,101,28,3,2024-11-15,Ravi,Yadav,user101@example.com,Delhi,2023-09-14,Smartwatch,Electronics,4999.0,Mega Plaza,Chennai,14997.0
3,1,102,11,2,2024-08-11,Nina,Joshi,user102@example.com,Mumbai,2024-01-21,Wireless Mouse,Electronics,799.99,Tech World Outlet,Bangalore,1599.98
4,1,103,18,3,2024-09-05,Sonal,Sharma,user103@example.com,Bangalore,2023-07-10,Wireless Mouse,Electronics,799.99,Downtown Mini Store,Pune,2399.9700000000003
3,3,104,13,4,2025-05-04,Karan,Patel,user104@example.com,Hyderabad,2024-02-05,Yoga Mat,Fitness,499.0,Tech World Outlet,Bangalore,1996.0
3,1,105,21,5,2024-10-02,Riya,Singh,user105@example.com,Chennai,2023-06-28,Wireless Mouse,Electronics,799.99,Tech World Outlet,Bangalore,3999.95
2,5,105,5,1,2025-03-17,Riya,Singh,user105@example.com,Chennai,2023-06-28,Notebook Set,Stationery,149.0,High Street Store,Delhi,149.0
4,3,105,2,5,2024-11-12,Riya,Singh,user105@example.com,Chennai,2023-06-28,Yoga Mat,Fitness,499.0,Downtown Mini Store,Pune,2495.0
3,9,107,22,4,2024-11-16,Priya,Kapoor,user107@example.com,Ahmedabad,2023-05-12,Dumbbell Set,Fitness,1999.0,Tech World Outlet,Bangalore,7996.0
1,5,108,12,4,2025-05-26,Rahul,Verma,user108@example.com,Kolkata,2023-08-19,Notebook Set,Stationery,149.0,City Mall Store,Mumbai,596.0
5,8,109,17,5,2024-07-10,Pooja,Mehta,user109@example.com,Delhi,2024-04-01,Desk Organizer,Accessories,399.0,Mega Plaza,Chennai,1995.0


In [0]:
from pyspark.sql.functions import sum, countDistinct, avg

gold_df = silver_df.groupBy(
    "transaction_date",
    "product_id", "product_name", "category",
    "store_id", "store_name", "location"
).agg(
    sum("quantity").alias("total_quantity_sold"),
    sum("total_amount").alias("total_sales_amount"),
    countDistinct("transaction_id").alias("number_of_transactions"),
    avg("total_amount").alias("average_transaction_value")
)


In [0]:
display(gold_df)

transaction_date,product_id,product_name,category,store_id,store_name,location,total_quantity_sold,total_sales_amount,number_of_transactions,average_transaction_value
2024-11-02,8,Desk Organizer,Accessories,1,City Mall Store,Mumbai,1,399.0,1,399.0
2024-08-11,1,Wireless Mouse,Electronics,3,Tech World Outlet,Bangalore,2,1599.98,1,1599.98
2024-12-13,8,Desk Organizer,Accessories,4,Downtown Mini Store,Pune,5,1995.0,1,1995.0
2025-05-04,3,Yoga Mat,Fitness,3,Tech World Outlet,Bangalore,4,1996.0,1,1996.0
2025-05-26,5,Notebook Set,Stationery,1,City Mall Store,Mumbai,4,596.0,1,596.0
2024-07-14,1,Wireless Mouse,Electronics,5,Mega Plaza,Chennai,1,799.99,1,799.99
2024-07-17,1,Wireless Mouse,Electronics,4,Downtown Mini Store,Pune,5,3999.95,1,3999.95
2024-09-05,1,Wireless Mouse,Electronics,4,Downtown Mini Store,Pune,3,2399.9700000000003,1,2399.9700000000003
2025-06-03,9,Dumbbell Set,Fitness,4,Downtown Mini Store,Pune,2,3998.0,1,3998.0
2024-08-27,2,Bluetooth Speaker,Electronics,2,High Street Store,Delhi,5,6497.45,1,6497.45


In [0]:
gold_path = "/mnt/retail/gold/"

gold_df.write.mode("overwrite").format("delta").save(gold_path)


In [0]:
spark.sql("""
CREATE TABLE retail_gold_sales_summary
USING DELTA
LOCATION '/mnt/retail/gold/' """)


Out[26]: DataFrame[]

In [0]:
%sql select * from retail_gold_sales_summary

transaction_date,product_id,product_name,category,store_id,store_name,location,total_quantity_sold,total_sales_amount,number_of_transactions,average_transaction_value
2024-11-02,8,Desk Organizer,Accessories,1,City Mall Store,Mumbai,1,399.0,1,399.0
2024-08-11,1,Wireless Mouse,Electronics,3,Tech World Outlet,Bangalore,2,1599.98,1,1599.98
2024-12-13,8,Desk Organizer,Accessories,4,Downtown Mini Store,Pune,5,1995.0,1,1995.0
2025-05-04,3,Yoga Mat,Fitness,3,Tech World Outlet,Bangalore,4,1996.0,1,1996.0
2025-05-26,5,Notebook Set,Stationery,1,City Mall Store,Mumbai,4,596.0,1,596.0
2024-07-14,1,Wireless Mouse,Electronics,5,Mega Plaza,Chennai,1,799.99,1,799.99
2024-07-17,1,Wireless Mouse,Electronics,4,Downtown Mini Store,Pune,5,3999.95,1,3999.95
2024-09-05,1,Wireless Mouse,Electronics,4,Downtown Mini Store,Pune,3,2399.9700000000003,1,2399.9700000000003
2025-06-03,9,Dumbbell Set,Fitness,4,Downtown Mini Store,Pune,2,3998.0,1,3998.0
2024-08-27,2,Bluetooth Speaker,Electronics,2,High Street Store,Delhi,5,6497.45,1,6497.45
